In [3]:
import fredapi as fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import requests
import os

# Set the API key

FRED_API_KEY = os.getenv('FRED_API_KEY')

# Create a FRED API object
fred_api = fred.Fred(api_key=FRED_API_KEY)

# Get the release dates for all releases
release_dates = fred_api.get_series_all_releases('UNRATE')

In [16]:
import sys

# 현재 작업 디렉토리 설정
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.append(project_root)

# 데이터 파일 경로 설정
data_path = os.path.join('../data', 'nowcast-model-variables.xlsx')

variables = pd.read_excel(data_path, engine='openpyxl', sheet_name='variables')
releases = pd.read_excel(data_path, engine='openpyxl', sheet_name='releases')

variables.to_csv('../data/nowcast-variables.csv')
releases.to_csv('../data/nowcast-releases.csv')

In [20]:
"""
데이터 수집 및 전처리를 위한 함수들
"""

import pandas as pd
import numpy as np
from tqdm import tqdm
import logging
from datetime import datetime
import os

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_fred_data(fred_api, series_id, start_date='2020-01-01', end_date=None):
    """
    FRED API를 통해 데이터를 가져오는 함수
    
    Parameters:
    -----------
    fred_api : fredapi.Fred
        FRED API 객체
    series_id : str
        FRED 시리즈 ID
    start_date : str
        시작 날짜 (YYYY-MM-DD 형식)
    end_date : str, optional
        종료 날짜 (YYYY-MM-DD 형식)
    
    Returns:
    --------
    pd.Series
        수집된 시계열 데이터
    """
    try:
        data = fred_api.get_series(series_id, observation_start=start_date, observation_end=end_date)
        return data
    except Exception as e:
        logger.error(f"FRED 데이터 수집 실패 - {series_id}: {str(e)}")
        return None

def get_yahoo_data(ticker, start_date='2020-01-01', end_date=None):
    """
    Yahoo Finance를 통해 데이터를 가져오는 함수
    
    Parameters:
    -----------
    ticker : str
        Yahoo Finance 티커 심볼
    start_date : str
        시작 날짜 (YYYY-MM-DD 형식)
    end_date : str, optional
        종료 날짜 (YYYY-MM-DD 형식)
    
    Returns:
    --------
    pd.DataFrame
        수집된 주가 데이터
    """
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        return data['Close']  # 종가만 반환
    except Exception as e:
        logger.error(f"Yahoo Finance 데이터 수집 실패 - {ticker}: {str(e)}")
        return None

def transform_data(data, transform_type):
    """
    데이터 변환 함수
    
    Parameters:
    -----------
    data : pd.Series
        원본 데이터
    transform_type : str
        변환 유형 ('dlog', 'pchg', 'apchg', 'base', 'diff1', 'log')
    
    Returns:
    --------
    pd.Series
        변환된 데이터
    """
    if data is None:
        return None
        
    if transform_type == 'dlog':
        return np.log(data).diff()
    elif transform_type == 'pchg':
        return data.pct_change()
    elif transform_type == 'apchg':
        return data.pct_change().diff()
    elif transform_type == 'base':
        return data
    elif transform_type == 'diff1':
        return data.diff()
    elif transform_type == 'log':
        return np.log(data)
    else:
        logger.warning(f"알 수 없는 변환 유형: {transform_type}")
        return data

def collect_all_data(variables_df, fred_api, start_date='2020-01-01', end_date=None):
    """
    모든 변수에 대한 데이터를 수집하는 함수
    
    Parameters:
    -----------
    variables_df : pd.DataFrame
        변수 정보가 담긴 데이터프레임
    fred_api : fredapi.Fred
        FRED API 객체
    start_date : str
        시작 날짜
    end_date : str, optional
        종료 날짜
    
    Returns:
    --------
    pd.DataFrame
        수집된 모든 데이터
    """
    collected_data = {}
    
    for _, row in tqdm(variables_df.iterrows(), total=len(variables_df), desc="데이터 수집 중"):
        varname = row['varname']
        source = row['hist_source']
        source_key = row['hist_source_key']
        transform = row['st']
        
        if source == 'fred':
            data = get_fred_data(fred_api, source_key, start_date, end_date)
        elif source == 'yahoo':
            data = get_yahoo_data(source_key, start_date, end_date)
        else:
            logger.warning(f"알 수 없는 데이터 소스: {source}")
            continue
            
        if data is not None:
            transformed_data = transform_data(data, transform)
            # 데이터가 2차원인 경우 1차원으로 변환
            if isinstance(transformed_data, pd.DataFrame):
                transformed_data = transformed_data.iloc[:, 0]
            collected_data[varname] = transformed_data
    
    # 데이터프레임 생성 전에 모든 시리즈의 인덱스를 통일
    common_index = None
    for series in collected_data.values():
        if common_index is None:
            common_index = series.index
        else:
            common_index = common_index.intersection(series.index)
    
    # 통일된 인덱스로 데이터프레임 생성
    aligned_data = {name: series[common_index] for name, series in collected_data.items()}
    return pd.DataFrame(aligned_data)

# 데이터 수집 실행
start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

collected_data = collect_all_data(variables, fred_api, start_date, end_date)

# 결과 저장
output_dir = '../data/collected'
os.makedirs(output_dir, exist_ok=True)
collected_data.to_csv(f'{output_dir}/collected_data_{datetime.now().strftime("%Y%m%d")}.csv')

# 데이터 확인
print("\n수집된 데이터 정보:")
print(collected_data.info())
print("\n결측치 개수:")
print(collected_data.isnull().sum())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
데이터 수집 중: 100%|██████████| 95/95 [00:27<00:00,  3.46it/s]


수집된 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 0 entries
Data columns (total 95 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mpce           0 non-null      float64
 1   po             0 non-null      float64
 2   ps             0 non-null      float64
 3   psr            0 non-null      float64
 4   pi             0 non-null      float64
 5   pid            0 non-null      float64
 6   bankci         0 non-null      float64
 7   bankre         0 non-null      float64
 8   delinqrer      0 non-null      float64
 9   delinqcc       0 non-null      float64
 10  delinqci       0 non-null      float64
 11  gdp            0 non-null      float64
 12  pce            0 non-null      float64
 13  pceg           0 non-null      float64
 14  pcegd          0 non-null      float64
 15  pcegdmotor     0 non-null      float64
 16  pcegdfurnish   0 non-null      float64
 17  pcegdrec       0 non-null      float64
 18  pcegdo